In [1]:
from sklearn.metrics import mean_absolute_error, r2_score
import lightgbm as lgb
import pandas as pd
import numpy as np
import pickle

In [2]:
seed = 100

In [3]:
folder_path = "../data/LightGBM/processed/"

In [6]:
with open(folder_path + "processed_outer_fold_0.pkl", "rb") as f:
    outer_fold = pickle.load(f)

    inner_fold = outer_fold['inner_folds'][0]

    # print(inner_fold['X_train'].columns)
# del outer_fold, inner_fold

In [9]:
X_train = inner_fold['X_train']
y_train = inner_fold['y_train']
X_val = inner_fold['X_val']
y_val = inner_fold['y_val']

 n_estimators, learning_rate, num_leaves, max_depth for the LGBM regressor model

 Key hyperparameters such as n_estimators (1000, 2000, 4000, 5000), learning_rate (0.01, 0.05, 0.1), num_leaves (100, 200, 300), max_depth (−1, 20, 30), min_child_sample (15, 20, 25) were optimized with randomized search. Prediction on the validation data set and the MSE values were monitored to ensure generalized learning and remove overfitting. If the loss (MSE) value for the prediction on the validation data set was not improved for 100 rounds early stopping was implemented and the final weight was retained to predict on the test data set.

In [ ]:
def train_lightgbm(X_train, y_train, X_val, y_val):
    """Train a LightGBM regressor and return model + validation MAE."""
    params = {
        "objective": "regression",
        "metric": "mae",
        "n_estimators": 5000,
        "learning_rate": 0.01,
        "num_leaves": 64,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "random_state": seed,
        "verbose": -1
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="mae",
        callbacks=[
            lgb.log_evaluation(period=0), # silence output of model
            lgb.early_stopping(stopping_rounds=50)
        ]
    )

    preds = model.predict(X_val)
    mae = mean_absolute_error(y_val, preds)
    r2 = r2_score(y_val, preds)

    return model, mae, r2

In [21]:
train_lightgbm(X_train, y_train, X_val, y_val)

Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[5000]	valid_0's l1: 0.166179


(LGBMRegressor(colsample_bytree=0.8, learning_rate=0.01, metric='mae',
               n_estimators=5000, num_leaves=64, objective='regression',
               random_state=100, subsample=0.8, verbose=-1),
 0.16617917748285765,
 0.9655709529072336)